In [43]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math
from dython.nominal import associations
from sklearn.preprocessing import MinMaxScaler
from functions import transform_raw_data

df = transform_raw_data(path_to_csv="stats19CycleCollisions2022.csv")
df = df.drop(columns=["accident_index_2"])

In [ ]:
# looking for values that should be classed as missing
for i in zip(df.columns, df.dtypes):
    if i[1] == 'object':
        print(i[0], ":\n ", df[i[0]].unique(), "\n")

In [3]:
# standardise missing values
missing_values = ['Data missing or out of range', 'unknown (self reported)', np.nan, 'Unknown', 'Not known', 'Undefined']
df = df.replace({i: "Missing" for i in missing_values})

In [ ]:
# column summaries
pd.DataFrame([i for i in zip(df.columns, df.dtypes, df.nunique(), ((df == "Missing") | (df.isnull())).sum(), 100 * ((df == "Missing") | (df.isnull())).mean())], columns=['column', 'dtype', 'nunique', 'missing', 'missing %'])

In [5]:
# drop lsoa variables - too many unique values
# keep longitude, latitude, date as I may use these in future to join to extra data, but do not use in model
# remove age band variables and keep age variables
# remove variables with a high % missing - special_conditions_at_site, carriageway_hazards, skidding_and_overturning, hit_object_in_carriageway, hit_object_off_carriageway, journey_purpose_of_driver
# (skidding_and_overturning, hit_object_in_carriageway, hit_object_off_carriageway are also variables that would only be known after the collision occurred - so would not be fair to include in the model)
# remove vehicle_leaving_carriageway for same reason above
df = df.drop(columns=[
    'lsoa_of_casualty', 'lsoa_of_driver', 
    'age_band_of_casualty', 'age_band_of_driver',
    'special_conditions_at_site', 'carriageway_hazards', 'skidding_and_overturning', 'hit_object_in_carriageway', 'hit_object_off_carriageway', 'journey_purpose_of_driver', 'vehicle_leaving_carriageway'
])

# categorise speed_limit
df['speed_limit'] = df['speed_limit'].astype('object')

# convert numeric to continuous
df[['engine_capacity_cc', 'age_of_casualty', 'age_of_driver']] = df[['engine_capacity_cc', 'age_of_casualty', 'age_of_driver']].replace('Missing', np.nan).astype('float')

In [ ]:
# check there are no duplicates
dups = df.duplicated()
dups.any()

In [ ]:
# summary statistics of continuous variables
df.describe()

In [8]:
# age_of_vehicle has values of -1 -> these must be missing values
df['age_of_vehicle'] = df['age_of_vehicle'].replace(-1, np.nan)

### dealing with missing values

In [ ]:
missing = pd.DataFrame([i for i in zip(df.columns, df.dtypes, df.nunique(), 100 * ((df == "Missing") | (df.isnull())).mean()) if i[3] > 0], columns=['column', 'dtype', 'nunique', 'missing %']).sort_values("missing %")
missing

categorical variables

In [ ]:
# viewing frequency distributions for categorical variables with missing values

# select only categorical columns
categorical_missing = list(missing[missing['dtype'] == 'object']['column'])

def plot_freq_dists(df):
    # calculate the number of rows and columns for a square layout
    num_vars = len(categorical_missing)
    cols = math.ceil(math.sqrt(num_vars))
    rows = math.ceil(num_vars / cols)

    # set up a grid of subplots
    fig, ax = plt.subplots(nrows=rows, ncols=cols, figsize=(5 * cols, 5 * rows))
    ax = ax.flatten()

    # trimming category labels to make the charts easier to read
    df_trimmed = df.applymap(lambda x: x[:10] if isinstance(x, str) else x)

    # plot frequency distribution for each categorical column
    for i, col in enumerate(categorical_missing):
        sns.countplot(x=col, data=df_trimmed, ax=ax[i], hue=col, legend=False)
        ax[i].set_title(f'Frequency of {col}')
        ax[i].xaxis.set_ticks(ax[i].get_xticks()) # add this line to avoid warning
        ax[i].set_xticklabels(ax[i].get_xticklabels(), rotation=45, ha='right')
        
    # hide any unused subplots
    for i in range(num_vars, len(ax)):
        fig.delaxes(ax[i])

    plt.tight_layout()
    plt.show()

plot_freq_dists(df)

- for missing vars we can either simple impute or use a more sophisticated method such as kNN
- for categorical vars we can also just leave as a "Missing" category

The STATS19 data is completed by police at the scene of the incident, so I don't think the missing data is inherently meaningful* for most variables i.e. it seems unlikely that police are deliberately not filling in certain fields because they don't want to share the information. It seems more likely that they simply forgot to fill it in, didn't have enough time, couldn't be bothered, the data was corrupted, etc. However, missing data could still be correlated with the casualty severity - I can imagine that police might be more inclined to spend time filling in all the fields on the form if the collision is not very serious and they have time to spare, however if there's a fatality then filling out a long form might be less of a priority. Since my primary goal is to infer the risk factors for cycle collision severity, I don't want to include these missing categories if they have this 'fake' correlation with casualty severity. I will investigate this further by looking at the correlation of each of these variables with the target, using Cramer's V.

*Edit: After reading the guidance to police for filling in the STATS19 form, it does actually seem that missing data can have some meaning. For example, if the sex of the driver is missing then this is likely because there was a hit-and-run incident and so the driver could not be identified. I think this backs up my choice to not add "Missing" as a separate category, as a hit-and-run incident is not something that could be known before the collision.

In [ ]:
associations(df[categorical_missing + ['casualty_severity']], nom_nom_assoc="cramer", figsize=(15, 15))

In [ ]:
# looking at associations again, but with missing values imputed while keeping the category distributions of each variable the same
df = df.replace("Missing", np.nan)

for col in categorical_missing:
    freq_dict = df[col].value_counts(normalize=True).to_dict()
    df[col] = df[col].fillna(pd.Series(np.random.choice(list(freq_dict.keys()), p=list(freq_dict.values()), size=len(df))))

# does not make sense for propulsion_code to have a value for cyclists
df["propulsion_code"] = np.where(df["vehicle_type"] == "Pedal cycle", "Undefined", df["propulsion_code"])   

associations(df[categorical_missing + ['casualty_severity']], nom_nom_assoc="cramer", figsize=(15, 15))

Lots of variables are highly associated with each other in the 1st grid but not at all in the 2nd grid - showing that they are only associated because of the missing values. Clearly these fields are either mostly filled in or not filled in at all. For example, in the 1st grid vehicle_left_hand_drive is strongly associated with vehicle_manoeuvre and vehicle_location_restricted_lane - and vehicle_left_hand_drive is also associated with casualty_severity. I suspect that the vehicle_left_hand_drive field is only ever filled in if the driver was doing something dangerous, like a U-turn or sitting in a restricted lane, which would explain why vehicle_left_hand_drive is associated with casualty_severity when I wouldn't really expect it to be.

In the 2nd grid the associations make more sense e.g. road_type and junction_detail, propulsion_code and vehicle_type. However, very few of the variables are associated with casualty_severity so I am not sure how useful they will be for the model.

In [ ]:
# plot frequency distributions with imputed missing values
plot_freq_dists(df)

continuous variables

In [ ]:
# plot box plots of each continous variable by target group

continuous_missing = list(missing[~(missing['dtype'] == 'object')]['column'])

def plot_continuous_vars(plot_type, vars, df):
    # set up the number of rows and columns for the grid
    num_vars = len(vars)
    num_cols = 2
    num_rows = (num_vars + num_cols - 1) // num_cols

    # create a figure and a grid of subplots
    fig, ax = plt.subplots(num_rows, num_cols, figsize=(15, num_rows * 5))
    ax = ax.flatten()

    for i, var in enumerate(vars):
        if plot_type == "boxplot":
            sns.boxplot(x="casualty_severity", y=var, data=df, ax=ax[i])
            ax[i].set_title(f'Boxplot of {var} by {"casualty_severity"}')
            ax[i].set_xlabel("casualty_severity")
            ax[i].set_ylabel(var)
        elif plot_type == "kde":
            sns.histplot(df[var], kde=True, ax=ax[i])
            ax[i].set_title(f'Histogram & KDE of {var}')
            ax[i].set_xlabel(var)
            ax[i].set_ylabel('Density')

    # remove empty axes
    for i in range(num_vars, num_rows * num_cols):
        fig.delaxes(ax[i])

    plt.tight_layout()
    plt.show()

plot_continuous_vars("boxplot", continuous_missing, df)

- engine_capacity_cc is much higher for the fatal group - not surprising but didn't expect this trend to be so clear
- clear trend in increasing age_of_casualty with increasing casualty severity - I was expecting younger people to be more at risk for fatal accidents due to them taking more risks on the bike, however this is showing the opposite. Perhaps because older people are more likely to take a bad fall, or are less confident on the road
- age_of_vehicle distribution looks no different between the different groups, except for a slightly shorter right-hand tail in the fatal group - median looks almost exactly the same between groups. The fatal group is smaller so could just be that the outliers in vehicle age happen to be in the other groups
- age_of_driver distribution for the fatal group has larger IQR but smaller range - larger IQR makes sense as we typically think of young or elderly drivers as being more dangerous

In [ ]:
plot_continuous_vars("kde", continuous_missing, df)

In [ ]:
# plotting correlation between all continuous variables and useful categorical variables (based on association grid above)
# can use this to see if any variables will be useful for imputing the continuous variables with missing values

def is_continuous(series):
    return pd.api.types.is_numeric_dtype(series)

# function assumes that continuous variables are listed before categorical columns in the 'columns' input
def plot_correlation_grid(df, columns, num_cols=3):
    num_vars = len(columns)
    num_continuous_vars = sum([0 if i in df.select_dtypes(include='object').columns else 1 for i in columns])
    num_plots = sum([num_vars - i for i in range(num_continuous_vars)])
    num_rows = (num_plots + num_cols - 1) // num_cols

    fig, ax = plt.subplots(num_rows, num_cols, figsize=(15, 50))
    ax = ax.flatten()
    
    # get all pairs of variables to plot
    plotted_pairs = 0
    max_plots = num_rows * num_cols

    # axes to delete
    delete_axes = []

    # trimming category labels to make the charts easier to read
    df_trimmed = df.applymap(lambda x: x[:10] if isinstance(x, str) else x)
    
    for i, var1 in enumerate(columns):
        for j, var2 in enumerate(columns):
            if i <= j:  # only plot unique combinations
                ax_sub = ax[plotted_pairs]
                
                if var1 == var2: # skip if same variable
                    ax_sub.axis('off') 
                    delete_axes.append(plotted_pairs)
                else:
                    # if both variables are continuous, create a scatter plot
                    if is_continuous(df_trimmed[var1]) and is_continuous(df_trimmed[var2]):
                        sns.scatterplot(x=df_trimmed[var1], y=df_trimmed[var2], ax=ax_sub)
                    # if one variable is continuous and the other is categorical, create a box plot
                    elif is_continuous(df_trimmed[var1]) and not is_continuous(df_trimmed[var2]):
                        sns.boxplot(x=df_trimmed[var2], y=df_trimmed[var1], ax=ax_sub)
                    elif is_continuous(df_trimmed[var2]) and not is_continuous(df_trimmed[var1]):
                        sns.boxplot(x=df_trimmed[var1], y=df_trimmed[var2], ax=ax_sub)
                    # skip plots for two categorical variables
                    else:
                        ax_sub.axis('off')
                        delete_axes.append(plotted_pairs)

                ax_sub.set_title(f'{var1} vs {var2}')
                ax_sub.xaxis.set_ticks(ax_sub.get_xticks()) # add this line to avoid warning
                ax_sub.set_xticklabels(ax_sub.get_xticklabels(), rotation=45, ha='right')
                plotted_pairs += 1
                
                if plotted_pairs >= max_plots:
                    break
        if plotted_pairs >= max_plots:
            break

    # delete empty axes
    for i in delete_axes:
        fig.delaxes(ax[i])
    
    plt.tight_layout()
    plt.show()

plot_correlation_grid(df, ['age_of_casualty','engine_capacity_cc','age_of_vehicle','age_of_driver',
                           'vehicle_type','junction_detail','towing_and_articulation','vehicle_manoeuvre','casualty_imd_decile','pedestrian_crossing_physical_facilities','driver_imd_decile'])

- strong correlation between engine_capacity_cc and vehicle_type and towing_and_articulation
- some correlation between age_of_vehicle and vehicle_type and towing_and_articulation
- age_of_casualty and age_of_driver also correlate to some extent with vehicle_type -> cannot think of a reason why age_of_casualty would be correlated with vehicle_type, so will ignore this correlation

I will drop age_of_vehicle since it has little correlation with casualty_severity anyway.

Therefore, I will impute each continuous variable based on the categorical variables that they are correlated with (using the median value of the category).

I'm using the target variable casualty_severity to impute some of the variables, so will need to make sure I apply the imputation to the training and test data separately in order to prevent data leakage.

In [17]:
df = df.drop(columns=["age_of_vehicle"])

In [18]:
# creating new vehicle_type column because some values of vehicle_type have no values for engine_capacity_cc -> using other types of vehicle that have the most similar engine size
df["vehicle_type_2"] = [
    "Goods over 3.5t. and under 7.5t" if vehicle_type in ["Agricultural vehicle", "Goods vehicle - unknown weight"]
    else "Motorcycle 50cc and under" if vehicle_type == "Electric motorcycle"
    else "Motorcycle 125cc and under" if vehicle_type == "Motorcycle - unknown cc"
    else "Car" if vehicle_type == "Unknown vehicle type (self rep only)"
    else vehicle_type
    for vehicle_type in df["vehicle_type"]
]

# imputing continuous variables
# first imputation should capture most nulls
# second imputation should capture any remaining nulls (remaining because there were no values in the lookup group)
impute_dict_1 = {
    "engine_capacity_cc": ["vehicle_type_2"],
    "age_of_casualty": ["casualty_severity"],
    "age_of_driver": ["vehicle_type", "casualty_severity"]
}
impute_dict_2 = {
    "engine_capacity_cc": ["vehicle_type_2"],
    "age_of_casualty": ["casualty_severity"],
    "age_of_driver": ["casualty_severity"]
}

def impute_continuous_vars(df, impute_dict):
    for var in impute_dict:
        lookup_df = df.groupby(impute_dict[var])[var].median()
        df = pd.merge(df, lookup_df, how="left", on=impute_dict[var], suffixes=["", "_y"])
        df[var] = np.where(df[var].isna(), df[var + '_y'], df[var])
        df = df.drop(columns=[var + '_y'])

    return df

df_1 = impute_continuous_vars(df, impute_dict_1)
df_2 = impute_continuous_vars(df_1, impute_dict_2)
df = df_2

# "Pedal cycle" has no values for engine_capacity_cc for obvious reasons
# assume average cyclist can push 100W ≈ 0.13 horsepower -> horsepower of standard car ~200 -> cyclist horsepower 0.2/200=0.00065 of a car -> set engine_capacity_cc of "Pedal cycle" to 0.00125 that of a car
engine_capacity_car = df.groupby("vehicle_type")["engine_capacity_cc"].median().loc["Car"]
df["engine_capacity_cc"] = np.where(df["vehicle_type"] == "Pedal cycle", 0.00065 * engine_capacity_car, df["engine_capacity_cc"])

df = df.drop(columns=["vehicle_type_2"])

In [19]:
# check there are no more missing values
assert sum(((df.isna()) | (df == "Missing")).any()) == 0

### creating new features

In [24]:
# creating date and time features
df['time_period'] = (df.time.str.slice(start=0, stop=2).astype('int') // 4)
df['time_period'] = (df['time_period'] * 4).astype('str') + ':00 - ' + ((df['time_period'] + 1) * 4).astype('str') + ':00'

df['date'] = pd.to_datetime(df.date)
df['month'] = df['date'].dt.month
df['season'] = [
    'spring' if month in [3, 4, 5]
    else 'summer' if month in [6, 7, 8]
    else 'autumn' if month in [9, 10, 11]
    else 'winter'
    for month
    in df['month']
]

df = df.drop(columns=['date', 'month'])

### transforming variables

In [ ]:
# plot continuous variable distributions
continuous_vars = [col for col in list(df.select_dtypes(exclude='object').columns) if col not in ['longitude', 'latitude']]
plot_continuous_vars('kde', continuous_vars, df)

- PRECTOTCORR and engine_capacity_cc are highly right skewed -> could apply a log or quantile transformation to these, but will leave for now
- age_of_driver distribution has been significantly affected by the imputation -> might experiment with dropping this variable as it does not seem very predictive anyway

Below I am normalising all continuous variables to fit into a [0, 1] range (I don't think this is necessary for the algorithms I'm using, but shouldn't do any harm), and one-hot encoding all categorical variables.

In [26]:
# normalise continuous variables
scaler = MinMaxScaler()
df[continuous_vars] = scaler.fit_transform(df[continuous_vars])

In [27]:
# one-hot encode categorical variables
categorical_vars = [col for col in list(df.select_dtypes(include='object').columns) if col not in ['date', 'time']]
df = pd.get_dummies(df, columns=categorical_vars, drop_first=False)

### Checking for multicollinearity